In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hemochromatosis"
cohort = "GSE159676"

# Input paths
in_trait_dir = "../../input/GEO/Hemochromatosis"
in_cohort_dir = "../../input/GEO/Hemochromatosis/GSE159676"

# Output paths
out_data_file = "../../output/preprocess/Hemochromatosis/GSE159676.csv"
out_gene_data_file = "../../output/preprocess/Hemochromatosis/gene_data/GSE159676.csv"
out_clinical_data_file = "../../output/preprocess/Hemochromatosis/clinical_data/GSE159676.csv"
json_path = "../../output/preprocess/Hemochromatosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Portal fibroblasts with mesenchymal stem cell features form a reservoir of proliferative myofibroblasts in liver fibrosis"
!Series_summary	"Based on the identification  of a transcriptomic signature, including Slit2,  characterizing portal mesenchymal stem cells  (PMSC)  and derived myofibroblast (MF), we examined  the gene expression profile of in liver tissue derived  from multiple human liver disorders, including  primary sclerosing cholangitis (PSC) (n=12), non-alcoholic steatohepatitis (NASH)  (n=7) and other liver diseases (i.e., primary biliary cholangitis, autoimmune hepatitis, alcoholic liver disease and haemochromatosis) (n=8) and  compared them to healthy controls (tumor free tissue from livers with metastasis from colorectal cancer) (n=5). We found that  SLIT2 was overexpressed in the liver of patients with NASH, PSC and other chronic liver diseases. We also examined the microarray data of the human liver tissue samples for the transcr

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import json
from typing import Callable, Optional, Dict, Any

# Define if gene expression data is available
is_gene_available = True  # Based on the background information, this dataset uses microarray data (Affymetrix Human Gene 1.0)

# Identify the row for trait data in sample characteristics dictionary
trait_row = 0  # The conditions including Hemochromatosis are in row 0

# Define data type conversion functions for trait (Hemochromatosis)
def convert_trait(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value indicates Hemochromatosis (case-insensitive)
    if 'haemochromatosis' in value.lower() or 'hemochromatosis' in value.lower():
        return 1
    else:
        return 0

# Age and gender data are not available in the provided sample characteristics
age_row = None
gender_row = None

def convert_age(value):
    return None  # Not needed as age data is not available

def convert_gender(value):
    return None  # Not needed as gender data is not available

# Determine trait availability
is_trait_available = trait_row is not None

# Initial validation and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If trait data is available, extract clinical features
if trait_row is not None:
    # Make sure the clinical_data directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    try:
        # Create a DataFrame from the sample characteristics dictionary
        # Using the sample characteristics dictionary provided in the previous step
        sample_char_dict = {0: ['condition: Liver tissue healthy', 'condition: Non alcoholic steatohepatitis', 
                              'condition: Primary sclerosing cholangitis', 'condition: Primary biliary cirrhosis', 
                              'condition: Haemochromatosis', 'condition: Autoimmune hepatitis', 'condition: Alcohol related']}
        
        # Convert the dictionary to a DataFrame
        clinical_data = pd.DataFrame()
        for key, values in sample_char_dict.items():
            clinical_data[key] = values
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"An error occurred when processing clinical data: {e}")
else:
    print("No trait data available, skipping clinical feature extraction.")


Preview of selected clinical features:
{0: [0.0]}
Clinical data saved to ../../output/preprocess/Hemochromatosis/clinical_data/GSE159676.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 17046 rows
First 20 gene IDs:
Index(['7896754', '7896759', '7896761', '7896779', '7896798', '7896817',
       '7896822', '7896859', '7896863', '7896865', '7896878', '7896882',
       '7896908', '7896917', '7896921', '7896929', '7896952', '7896983',
       '7896985', '7897026'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown are numeric IDs (like '7896754') which are not standard human gene symbols.
# Human gene symbols typically follow patterns like 'BRCA1', 'TP53', etc.
# These appear to be probe IDs or some other platform-specific identifiers that would need mapping.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE159676
Line 6: !Series_title = Portal fibroblasts with mesenchymal stem cell features form a reservoir of proliferative myofibroblasts in liver fibrosis
Line 7: !Series_geo_accession = GSE159676
Line 8: !Series_status = Public on Oct 21 2020
Line 9: !Series_submission_date = Oct 20 2020
Line 10: !Series_last_update_date = Mar 16 2022
Line 11: !Series_pubmed_id = 35278227
Line 12: !Series_summary = Based on the identification  of a transcriptomic signature, including Slit2,  characterizing portal mesenchymal stem cells  (PMSC)  and derived myofibroblast (MF), we examined  the gene expression profile of in liver tissue derived  from multiple human liver disorders, including  primary sclerosing cholangitis 


Gene annotation preview:
{'ID': [7896736, 7896738, 7896740, 7896742, 7896744], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7, 31, 24, 6, 36], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST000

### Step 6: Gene Identifier Mapping

In [7]:
# 1. First, observe the gene identifiers in gene_data and find matching identifiers in gene annotation
print("Gene identifiers in gene_data are numerical IDs like:", gene_data.index[:5])
print("\nExamining gene annotation columns to find appropriate mappings:")
for col in gene_annotation.columns:
    print(f"Column '{col}': {gene_annotation[col].iloc[0]}")

# Based on observation, the 'ID' column in gene_annotation matches our gene_data index format
# The 'gene_assignment' column contains gene information including gene symbols

# 2. Create mapping dataframe with ID and Gene columns
# Use the built-in extract_human_gene_symbols function which is designed to parse gene symbols
mapping_df = pd.DataFrame({
    'ID': gene_annotation['ID'].astype(str),
    'Gene': gene_annotation['gene_assignment'].apply(extract_human_gene_symbols)
})

print("\nPreview of mapping dataframe before filtering:")
print(mapping_df.head())
print(f"Total mappings before filtering: {len(mapping_df)}")

# Filter out empty gene lists
mapping_df = mapping_df[mapping_df['Gene'].apply(lambda x: len(x) > 0)]
print(f"Total mappings after filtering empty genes: {len(mapping_df)}")

# Print the first few rows with genes to verify extraction
print("\nSample of successfully extracted genes:")
sample_genes = mapping_df.head(5)
for idx, row in sample_genes.iterrows():
    print(f"ID: {row['ID']}, Genes: {row['Gene']}")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("\nPreview of gene expression data after mapping:")
print(f"Number of unique genes: {len(gene_data.index)}")
if len(gene_data.index) > 0:
    print("First 10 gene symbols:")
    print(gene_data.index[:10])
    # Save the gene data to file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
else:
    print("No genes were mapped! Checking for potential issues...")
    
    # Additional diagnostics
    print("\nChecking if gene_data indices exist in mapping_df:")
    sample_indices = list(gene_data.index[:5])
    for idx in sample_indices:
        print(f"Index {idx} exists in mapping_df: {idx in mapping_df['ID'].values}")
    
    # Try an alternative approach with a more direct mapping
    print("\nAttempting alternative mapping approach...")
    direct_mapping = {}
    for idx, row in mapping_df.iterrows():
        probe_id = row['ID']
        genes = row['Gene']
        if genes:
            for gene in genes:
                if probe_id not in direct_mapping:
                    direct_mapping[probe_id] = []
                direct_mapping[probe_id].append(gene)
    
    # Check if we have mappings for some of our gene_data indices
    print(f"Direct mapping created with {len(direct_mapping)} entries")
    for idx in sample_indices:
        print(f"Probe {idx} maps to: {direct_mapping.get(idx, 'No mapping')}")
    
    # Create a new mapping dataframe and try again
    new_mapping_rows = []
    for probe_id, genes in direct_mapping.items():
        new_mapping_rows.append({'ID': probe_id, 'Gene': genes})
    
    if new_mapping_rows:
        new_mapping_df = pd.DataFrame(new_mapping_rows)
        print(f"Created new mapping dataframe with {len(new_mapping_df)} rows")
        
        # Try applying gene mapping again
        gene_data_alt = apply_gene_mapping(gene_data, new_mapping_df)
        print(f"Alternative approach results: {len(gene_data_alt.index)} genes mapped")
        
        if len(gene_data_alt.index) > 0:
            print("First 10 gene symbols from alternative approach:")
            print(gene_data_alt.index[:10])
            gene_data = gene_data_alt  # Use the alternative results
            
            # Save the gene data to file
            os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
            gene_data.to_csv(out_gene_data_file)
            print(f"Gene expression data saved to {out_gene_data_file}")
        else:
            print("Alternative approach also failed to map genes")
            # Save empty dataframe as fallback
            gene_data.to_csv(out_gene_data_file)
            print(f"Empty gene expression data saved to {out_gene_data_file}")
    else:
        print("Could not create alternative mapping")
        # Save empty dataframe as fallback
        gene_data.to_csv(out_gene_data_file)
        print(f"Empty gene expression data saved to {out_gene_data_file}")

print("\nShape of gene expression data:", gene_data.shape)


Gene identifiers in gene_data are numerical IDs like: Index(['7896754', '7896759', '7896761', '7896779', '7896798'], dtype='object', name='ID')

Examining gene annotation columns to find appropriate mappings:
Column 'ID': 7896736
Column 'GB_LIST': nan
Column 'SPOT_ID': chr1:53049-54936
Column 'seqname': chr1
Column 'RANGE_GB': NC_000001.10
Column 'RANGE_STRAND': +
Column 'RANGE_START': 53049
Column 'RANGE_STOP': 54936
Column 'total_probes': 7
Column 'gene_assignment': ---
Column 'mrna_assignment': NONHSAT060105 // NONCODE // Non-coding transcript identified by NONCODE // chr1 // 100 // 100 // 7 // 7 // 0
Column 'category': main



Preview of mapping dataframe before filtering:
        ID                                               Gene
0  7896736                                                 []
1  7896738                          [OR4G2P, OR4G11P, OR4G1P]
2  7896740  [OR4F4, OR4F17, OR4F5, BC136848, BC136867, BC1...
3  7896742  [F-, AK298283, AL137655, BC032332, PCMTD2, L-,...
4  7896744  [OR4F29, OR4F3, OR4F16, OR4F21, OR4F7P, OR4F1P...
Total mappings before filtering: 33297
Total mappings after filtering empty genes: 25127

Sample of successfully extracted genes:
ID: 7896738, Genes: ['OR4G2P', 'OR4G11P', 'OR4G1P']
ID: 7896740, Genes: ['OR4F4', 'OR4F17', 'OR4F5', 'BC136848', 'BC136867', 'BC136907', 'BC136908']
ID: 7896742, Genes: ['F-', 'AK298283', 'AL137655', 'BC032332', 'PCMTD2', 'L-', 'D-', 'O-', 'BC118988', 'BC122537', 'BC131690', 'SEPT14', 'AK301928']
ID: 7896744, Genes: ['OR4F29', 'OR4F3', 'OR4F16', 'OR4F21', 'OR4F7P', 'OR4F1P', 'OR4F2P', 'OR4F28P', 'BC137547', 'BC137568', 'OR4F8P']
ID: 7896746, Gene

Direct mapping created with 25127 entries
Created new mapping dataframe with 25127 rows
Alternative approach results: 0 genes mapped
Alternative approach also failed to map genes
Empty gene expression data saved to ../../output/preprocess/Hemochromatosis/gene_data/GSE159676.csv

Shape of gene expression data: (0, 33)


### Step 7: Data Normalization and Linking

### Step 8: Data Normalization and Linking

In [8]:
import numpy as np
import os
import gzip

# 1. Extract gene expression data using the alternative approach that worked in Step 3
with gzip.open(matrix_file, 'rt') as file:
    # Find the start of the data
    for line in file:
        if '!series_matrix_table_begin' in line:
            break
            
    # Read the headers and data
    gene_data = pd.read_csv(file, sep='\t', index_col=0)

# Check if we have gene data before proceeding
if gene_data.empty:
    print("No gene expression data found in the matrix file.")
    is_gene_available = False
else:
    is_gene_available = True
    print(f"Gene data shape before normalization: {gene_data.shape}")

    # Normalize gene symbols using the NCBI Gene database information
    try:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
        
        # Save the normalized gene data to the output file
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        normalized_gene_data.to_csv(out_gene_data_file)
        print(f"Normalized gene data saved to {out_gene_data_file}")
    except Exception as e:
        print(f"Error normalizing gene data: {e}")
        is_gene_available = False
        normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Link clinical and genetic data
# In Step 2, we determined that Height data is not available in this dataset (trait_row = None)
# Create a minimal clinical dataframe with the trait column (containing NaNs)
if is_gene_available:
    sample_ids = gene_data.columns
    minimal_clinical_df = pd.DataFrame(index=sample_ids)
    minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values

    # If we have age and gender data from Step 2, add those columns
    if age_row is not None:
        minimal_clinical_df['Age'] = get_feature_data(clinical_data, age_row, 'Age', convert_age).iloc[0]

    if gender_row is not None:
        minimal_clinical_df['Gender'] = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender).iloc[0]

    minimal_clinical_df.index.name = 'Sample'

    # Save this minimal clinical data for reference
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    minimal_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

    # Create a linked dataset 
    if is_gene_available and normalized_gene_data is not None:
        linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
        linked_data.index.name = 'Sample'
        print(f"Linked data shape: {linked_data.shape}")
    else:
        linked_data = minimal_clinical_df
        print("No gene data to link with clinical data.")
else:
    # Create a minimal dataframe with just the trait for the validation step
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No gene data available, creating minimal dataframe for validation.")

# 4 & 5. Validate and save cohort information
# Since trait_row was None in Step 2, we know Height data is not available
is_trait_available = False  # Height data is not available

note = "Dataset contains gene expression data but no Height measurements. This dataset is not usable for studying Height associations."

# For datasets without trait data, we set is_biased to False
# This indicates the dataset is not usable due to missing trait data, not due to bias
is_biased = False

# Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Since there is no trait data, the dataset is not usable for our association study
# So we should not save it to out_data_file
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset does not contain Height data and cannot be used for association studies.")

Gene data shape before normalization: (17047, 33)
Gene data shape after normalization: (0, 33)
Normalized gene data saved to ../../output/preprocess/Hemochromatosis/gene_data/GSE159676.csv
Clinical data saved to ../../output/preprocess/Hemochromatosis/clinical_data/GSE159676.csv
Linked data shape: (33, 1)
Abnormality detected in the cohort: GSE159676. Preprocessing failed.
A new JSON file was created at: ../../output/preprocess/Hemochromatosis/cohort_info.json
Dataset usability: False
Dataset does not contain Height data and cannot be used for association studies.


/tmp/ipykernel_39342/939407384.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_data = pd.read_csv(file, sep='\t', index_col=0)
